# Entity matching

My baseline will be comparing embeddings of ORG chunks and company names.

The large spacy model already provides embedding functionality, as well as comparison.

For the embeddings matching I will use faiss

In [ ]:
import spacy

In [22]:
nlp = spacy.load("ru_core_news_lg")
text = "Сбербанк рассматривает возможность перехода в частную собственность, чтобы успокоить Пекин".lower()
doc = nlp(text)

In [23]:
for token in doc:
    print(token.text, token.has_vector)

сбербанк True
рассматривает True
возможность True
перехода True
в True
частную True
собственность True
, False
чтобы True
успокоить True
пекин True


In [24]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_, ent.has_vector)

сбербанк 0 8 ORG True
пекин 85 90 LOC True


In [27]:
doc[0].vector.shape

(300,)

## FAISS

In [28]:
import numpy as np
import faiss 

d = 300                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries

xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

                  
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

k = 1                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # I - indices, D - distances
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

True
100000
[[0]
 [1]
 [2]
 [3]
 [4]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[1039]
 [ 774]
 [  33]
 [ 426]
 [ 258]]
[[10957]
 [10073]
 [ 9788]
 [10911]
 [10095]]


1. В индексе правильные вектора
2. Находим вектор для компании, ищем ближайший в faiss
3. 